```{contents}
```

## Backward & Forward Propogation with RNN context

RNNs are used for **sequential data** — text, speech, time series, etc.

They process inputs **one at a time**, keeping **memory** of previous steps through **hidden states**.

The **training** of an RNN has two major parts:

1. **Forward Propagation** → compute outputs step by step through time.
2. **Backward Propagation Through Time (BPTT)** → compute gradients and update weights.

---

### Notation

At time step $t$:

| Symbol       | Description                                |
| ------------ | ------------------------------------------ |
| $x_t$      | Input vector                               |
| $h_t$      | Hidden state (memory)                      |
| $y_t$      | Output                                     |
| $W_x$      | Input → hidden weights                     |
| $W_h$      | Hidden → hidden weights                    |
| $W_y$      | Hidden → output weights                    |
| $b_h, b_y$ | Bias terms                                 |
| $f$        | Activation function (usually tanh or ReLU) |
| $g$        | Output activation (sigmoid or softmax)     |

---

###  Forward Propagation (Step-by-Step)

#### Equation 1 — Hidden State Update

$$
h_t = f(W_x x_t + W_h h_{t-1} + b_h)
$$
Each new hidden state $h_t$ depends on:

* Current input $x_t$
* Previous hidden state $h_{t-1}$

This allows **temporal context**.

---

### Equation 2 — Output

$$
y_t = g(W_y h_t + b_y)
$$
This gives the model’s prediction at time step $t$.

---

### Example Flow (Sequence Length = 3)

```
x1 → h1 → y1
      ↑
x2 → h2 → y2
      ↑
x3 → h3 → y3
```

**Operations:**

$$
\begin{align*}
h_1 &= f(W_x x_1 + W_h h_0 + b_h) \
h_2 &= f(W_x x_2 + W_h h_1 + b_h) \
h_3 &= f(W_x x_3 + W_h h_2 + b_h) \
y_3 &= g(W_y h_3 + b_y)
\end{align*}
$$

---

### Loss Computation

If there is a loss at each step:

$$
L = \sum_{t=1}^T \ell(y_t, \hat{y_t})
$$

If only final output matters (e.g. classification):
$$
L = \ell(y_T, \hat{y_T})
$$

---

### Backward Propagation (Backpropagation Through Time — BPTT)

Now, we update the parameters using gradient descent.

Because $h_t$ depends on all previous hidden states, **we must propagate gradients backward through time**.

---

### Step 1 — Unroll the RNN

Unrolling means visualizing the recurrence as a deep feedforward network through time steps:

```
x1 → h1 → y1
x2 → h2 → y2
x3 → h3 → y3
```

Here, **the same weights (W_x, W_h, W_y)** are shared across all time steps.

---

### Compute Gradients Backward

We compute gradients in **reverse order (from last time step to first)**.

At time $t$:

$$
\frac{\partial L}{\partial W_x}, \quad
\frac{\partial L}{\partial W_h}, \quad
\frac{\partial L}{\partial W_y}
$$

We use the **chain rule** because each hidden state depends on the previous one.

---

### Step 3 — Chain Rule Dependency

For the output layer:

$$
\frac{\partial L_t}{\partial W_y} = \frac{\partial L_t}{\partial y_t} \cdot \frac{\partial y_t}{\partial W_y}
$$

For the hidden state:

$$
\frac{\partial L_t}{\partial h_t} =
\frac{\partial L_t}{\partial y_t} \cdot \frac{\partial y_t}{\partial h_t}

* \frac{\partial L_{t+1}}{\partial h_{t+1}} \cdot \frac{\partial h_{t+1}}{\partial h_t}
$$

That second term accounts for **future dependencies** — the core reason it’s called *backpropagation through time.*

---

### Step 4 — Weight Updates

After computing the gradients, parameters are updated via gradient descent:
$$
W \leftarrow W - \eta \frac{\partial L}{\partial W}
$$

Where $\eta$ is the learning rate.

---

### Mathematical Intuition of Gradient Flow

Unrolling an RNN across $T$ steps gives this recursive gradient structure:

$$
\frac{\partial L}{\partial W_h} = \sum_{t=1}^T \frac{\partial L}{\partial h_t} \frac{\partial h_t}{\partial W_h}
$$
and
$$
\frac{\partial h_t}{\partial h_{t-1}} = W_h^T f'(a_{t-1})
$$

So gradient depends on repeated multiplication of $W_h$ and derivative $f'$.

If $||W_h|| < 1$ repeatedly, gradients **vanish** → network “forgets” long-term dependencies.
If $||W_h|| > 1$, gradients **explode** → unstable training.

---

### Vanishing and Exploding Gradient Problem

RNNs suffer from these because:
$$
\frac{\partial L}{\partial h_t} \propto \prod_{k=t}^{T} W_h^T f'(a_k)
$$

When $T$ (sequence length) is large, this product can:

* Shrink to zero (vanishing gradients)
* Blow up to infinity (exploding gradients)

This is why **LSTM** and **GRU** were developed — to regulate how much past information is retained or forgotten.

---

### Summary of Key Equations

### Forward Pass

$$
\begin{align*}
h_t &= f(W_x x_t + W_h h_{t-1} + b_h) \
y_t &= g(W_y h_t + b_y)
\end{align*}
$$

### Backward Pass

$$
\begin{align*}
\frac{\partial L}{\partial W_y} &= \sum_t \frac{\partial L_t}{\partial y_t} \frac{\partial y_t}{\partial W_y} \
\frac{\partial L}{\partial W_x} &= \sum_t \frac{\partial L_t}{\partial h_t} \frac{\partial h_t}{\partial W_x} \
\frac{\partial L}{\partial W_h} &= \sum_t \frac{\partial L_t}{\partial h_t} \frac{\partial h_t}{\partial h_{t-1}} \frac{\partial h_{t-1}}{\partial W_h}
\end{align*}
$$

---

**Intuitive Summary**

| Step                 | Purpose                                   | Concept                             |
| -------------------- | ----------------------------------------- | ----------------------------------- |
| Forward pass         | Compute predictions                       | Combine past + current info         |
| Compute loss         | Measure error                             | $L = \sum_t \ell(y_t, \hat{y_t})$ |
| Backward pass (BPTT) | Propagate gradients backward through time | Adjust weights for all time steps   |
| Weight update        | Optimize via gradient descent             | Learn temporal patterns             |

---

**In Short**

* **Forward propagation:** moves information forward through time.
  It updates hidden states using current input + previous state.

* **Backward propagation:** moves gradients backward through time.
  It updates weights by linking all time steps via the chain rule.